In [1]:
import googlemaps
from datetime import datetime, timedelta
from pprint import pprint
import pytz
import os
import json
import gmaps as gmv
import numpy as np
import itertools

Load configuration

In [2]:
config_file = os.path.join('C:\\', 'Users', 'glenn', 'src', 'pycommute', 'config.json')
with open(config_file, 'r') as f:
    config = json.load(f)

Find next Monday morning

In [3]:
local_tz = pytz.timezone(config['time_zone'])
arrival_time = datetime.now(local_tz).replace(hour=8, minute=0, second=0, microsecond=0) + timedelta(days=1) # Tomorrow at 8
while arrival_time.weekday() != 0: # Finding the next Monday.
    arrival_time += timedelta(days=1)
    
print(f'Calculating for arrival at {arrival_time}.')

Calculating for arrival at 2020-02-24 08:00:00+01:00.


Instantiate API

In [4]:
gmd = googlemaps.Client(key=config['api_key']) # Data API
gmv.configure(api_key=config['api_key']) # Visualization API

Convert location strings to lat/lon

In [5]:
origins_geocodes = [gmd.geocode(origin)[0] for origin in config['origins']]
origins_pos = [(gc['geometry']['location']['lat'], gc['geometry']['location']['lng']) for gc in origins_geocodes]
destinations_geocodes = [gmd.geocode(destination)[0] for destination in config['destinations']]
destinations_pos = [(gc['geometry']['location']['lat'], gc['geometry']['location']['lng']) for gc in destinations_geocodes]

Visualize the area

In [6]:
zoom_level = 9.5
center = 59.84, 10.80
fig = gmv.figure(center=center, zoom_level=zoom_level, map_type='ROADMAP')
fig

Figure(layout=FigureLayout(height='420px'))

Call distance matrix API

In [7]:
args = {
    'mode': 'transit',
    'language': config.get('language', 'english'),
    'units': config.get('units', 'metric'),
    'origins': origins_pos,
    'destinations': destinations_pos,
    'arrival_time': arrival_time,
}

distance_matrix = gmd.distance_matrix(**args)

Create matrix of durations for heatmap

In [8]:
transit_durations = [[row['elements'][i]['duration']['value'] for row in distance_matrix['rows']] for i in range(len(destinations_pos))]

Create heatmap

In [9]:
heatmap_layer = gmv.heatmap_layer(locations=origins_pos, weights=transit_durations[0])
fig = gmv.figure(center=center, zoom_level=zoom_level, map_type='ROADMAP')
#fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

Generate grid of positions

Define function that finds the latitude and longitude of a point a distance `d` from a point.

In [10]:
def point_from_distance(lat0d, lon0d, distance, heading):
    if heading == 'north':
        psi = 0
    elif heading == 'east':
        psi = -np.pi/2
    elif heading == 'south':
        psi = np.pi
    elif heading == 'west':
        psi = np.pi/2
    else:
        psi = np.radians(heading)
    
    G = 6371000 # great circle radius
    d = distance / G
    
    lat0 = np.radians(lat0d)
    lon0 = np.radians(lon0d)
    
    lat = np.arcsin(np.sin(lat0) * np.cos(d) + np.cos(lat0) * np.sin(d) * np.cos(psi))
    if abs(np.cos(lat)) <= 0.001:
        lon = lon0
    else:
        lon = ((lon0 - np.arcsin(np.sin(psi) * np.sin(d) / np.cos(lat)) + np.pi) % (2*np.pi)) - np.pi
    
    return np.degrees(lat), np.degrees(lon)

Create grid

In [11]:
southwest = 59.660417, 10.351575
northeast = 60.024253, 11.345597

lat_center = (northeast[0]+southwest[0])/2
lon_center = (northeast[1]+southwest[1])/2

dist = 2000

dlat = point_from_distance(lat_center, lon_center, dist, 'north')[0] - lat_center
dlon = point_from_distance(lat_center, lon_center, dist, 'east')[1] - lon_center

latitudes = np.arange(southwest[0], northeast[0], dlat)
longitudes = np.arange(southwest[1], northeast[1], dlon)

positions = list(itertools.product(latitudes, longitudes))
print(len(positions))

588


Split into eatable pieces

In [12]:
L = 25
ipositions = iter(positions)
positions_batches = []
while True:
    next_batch = list(itertools.islice(ipositions, L))
    if not next_batch:
        break
    positions_batches.append(next_batch)

Get distances

In [13]:
static_args = {
    'mode': 'transit',
    'language': config.get('language', 'english'),
    'units': config.get('units', 'metric'),
    'destinations': destinations_pos,
    'arrival_time': arrival_time,
}

distance_matrices = []
for positions_batch in positions_batches:
    distance_matrix = gmd.distance_matrix(origins=positions_batch, **static_args)
    distance_matrices.append(distance_matrix)


Concatenate into one distance matrix

In [14]:
i_distance_matrix = iter(distance_matrices)
distance_matrix = next(i_distance_matrix)
for matrix in i_distance_matrix:
    distance_matrix['origin_addresses'].extend(matrix['origin_addresses'])
    distance_matrix['rows'].extend(matrix['rows'])

Create heatmap layer

In [15]:
transit_durations = np.array([[row['elements'][i].get('duration', {'value': np.nan})['value']/60 for row in distance_matrix['rows']] for i in range(len(destinations_pos))])
heatmap_layer = gmv.heatmap_layer(locations=positions, weights=np.max(transit_durations,0), max_intensity=60)
fig = gmv.figure(center=center, zoom_level=12, map_type='ROADMAP')
#fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))